In [1]:
from io import BytesIO
import lxml.html
import pandas as pd
import requests

In [2]:
# Constants
REQUEST_URL = "https://bulbapedia.bulbagarden.net/w/api.php?action=parse&format=json&page=List_of_Pok%C3%A9mon_Trading_Card_Game_expansions"

In [3]:
# graceful fetch - error handling to be dealt properly in implementation
def fetch_contents(uri):
    try:
        response = requests.get(uri, timeout=5)
        response.raise_for_status()
    
        # if successful
        return response
    except requests.exceptions.HTTPError as errh:
        print(errh)
    except requests.exceptions.ConnectionError as errc:
        print(errc)
    except requests.exceptions.Timeout as errt:
        print(errt)
    except requests.exceptions.RequestException as err:
        print(err)

In [4]:
# fetch and save to file
response = fetch_contents(REQUEST_URL)
# file_representation = BytesIO(html_content)
with open("temp.txt", "wb") as fd:
    for chunk in response.iter_content(chunk_size=128):
        fd.write(chunk)

In [7]:
# successful experiment: replace the offending colspans, and convert from string to int
with open("temp.txt", "r") as fd:
    line = fd.readline()
    
    print(line.count('colspan=\\"2\\"'))
    new_line = line.replace('colspan=\\"2\\"', "colspan=2")
    print(new_line.count('colspan=\\"2\\"'))

    print(new_line.count("\\n"))
    new_line = new_line.replace("\\n", "")
    print(new_line.count("\\n"))

    with open("temp2.txt", "w") as fd:
        fd.write(new_line)

3
0
3184
0


In [8]:
# debug print
def print_all_tables(file):
    tables = pd.read_html(file)
    
    for index, table in enumerate(tables):
        print("---------------------------------------------------------------\n")
        print(f"Table No. {index}:")
        print(table)

print_all_tables("temp2.txt")

---------------------------------------------------------------

Table No. 0:
   Set no.  Symbol Logo of Expansion Name of Expansion      Type of Expansion  \
0        1  \u2014            \u2014          Base Set  Main Series Expansion   
1        2     NaN               NaN            Jungle  Main Series Expansion   
2        3     NaN               NaN            Fossil  Main Series Expansion   
3        4     NaN               NaN        Base Set 2  Main Series Expansion   
4        5     NaN               NaN       Team Rocket  Main Series Expansion   
5        6     NaN               NaN        Gym Heroes  Main Series Expansion   
6        7     NaN               NaN     Gym Challenge  Main Series Expansion   

       No. of cards       Release date Set abb.  
0               102    January 9, 1999       BS  
1                64      June 16, 1999       JU  
2                62   October 10, 1999       FO  
3               130  February 24, 2000       B2  
4  82 1 Secret card    

In [9]:
# read file post-edit
tables = pd.read_html("temp2.txt")
table = tables[0]
last_table = tables[-1]

In [ ]:
# debug prints
if "Set abb.\\n" in table.columns:
    print("found")
else:
    print("not found")
if "Set abb.\\n" in last_table.columns:
    print("found")
else:
    print("not found")

In [19]:
# Extract only desired tables
filtered_tables = [df for df in tables if "Set abb." in df.columns]

In [20]:
# pretty print (debug)
for index, df in enumerate(filtered_tables):
    print("---------------------------------------------------------------\n")
    print(f"Table No. {index}:")
    print(df)

---------------------------------------------------------------

Table No. 0:
   Set no.  Symbol Logo of Expansion Name of Expansion      Type of Expansion  \
0        1  \u2014            \u2014          Base Set  Main Series Expansion   
1        2     NaN               NaN            Jungle  Main Series Expansion   
2        3     NaN               NaN            Fossil  Main Series Expansion   
3        4     NaN               NaN        Base Set 2  Main Series Expansion   
4        5     NaN               NaN       Team Rocket  Main Series Expansion   
5        6     NaN               NaN        Gym Heroes  Main Series Expansion   
6        7     NaN               NaN     Gym Challenge  Main Series Expansion   

       No. of cards       Release date Set abb.  
0               102    January 9, 1999       BS  
1                64      June 16, 1999       JU  
2                62   October 10, 1999       FO  
3               130  February 24, 2000       B2  
4  82 1 Secret card    

In [21]:
name_and_set = [df[["Name of Expansion", "Set abb."]] for df in filtered_tables]
merged = pd.concat(name_and_set, ignore_index=True)

# pretty print (debug)
for index, df in enumerate(merged):
    print("---------------------------------------------------------------\n")
    print(f"Table No. {index}:")
    print(df)

---------------------------------------------------------------

Table No. 0:
Name of Expansion
---------------------------------------------------------------

Table No. 1:
Set abb.


In [ ]:
# debug
for index, df in enumerate(filtered_tables):
    try:
        df[["Name of Expansion", "Set abb."]]
    except:
        print(f"Index: {index}")
        print(df.columns)

In [25]:
print(merged.info(verbose=True))
print(type(merged))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Name of Expansion  152 non-null    object
 1   Set abb.           152 non-null    object
dtypes: object(2)
memory usage: 2.5+ KB
None
<class 'pandas.core.frame.DataFrame'>


In [28]:
print(merged[merged["Set abb."] != "\\u2014"])

                          Name of Expansion Set abb.
0                                  Base Set       BS
1                                    Jungle       JU
2                                    Fossil       FO
3                                Base Set 2       B2
4                               Team Rocket       TR
..                                      ...      ...
146                        Southern Islands       SI
148             Pok\u00e9 Card Creator Pack       CC
149                     Pok\u00e9mon Rumble       RM
150                     Pok\u00e9mon Futsal    FUT20
151  Pok\u00e9mon Trading Card Game Classic       CL

[151 rows x 2 columns]


In [29]:
final = merged[merged["Set abb."] != "\\u2014"].rename(columns={"Name of Expansion": "full_name", "Set abb.": "set_abb"})
for row in final.itertuples(index=False):
    print(row.set_abb)
    print(row.full_name)

BS
Base Set
JU
Jungle
FO
Fossil
B2
Base Set 2
TR
Team Rocket
G1
Gym Heroes
G2
Gym Challenge
N1
Neo Genesis
N2
Neo Discovery
N3
Neo Revelation
N4
Neo Destiny
LC
Legendary Collection
EX
Expedition Base Set
AQ
Aquapolis
SK
Skyridge
RS
EX Ruby & Sapphire
SS
EX Sandstorm
DR
EX Dragon
MA
EX Team Magma vs Team Aqua
HL
EX Hidden Legends
FL
EX FireRed & LeafGreen
TRR
EX Team Rocket Returns
DX
EX Deoxys
EM
EX Emerald
UF
EX Unseen Forces
DS
EX Delta Species
LM
EX Legend Maker
HP
EX Holon Phantoms
CG
EX Crystal Guardians
DF
EX Dragon Frontiers
PK
EX Power Keepers
DP
Diamond & Pearl
MT
Diamond & Pearl\u2014Mysterious Treasures
SW
Diamond & Pearl\u2014Secret Wonders
GE
Diamond & Pearl\u2014Great Encounters
MD
Diamond & Pearl\u2014Majestic Dawn
LA
Diamond & Pearl\u2014Legends Awakened
SF
Diamond & Pearl\u2014Stormfront
PL
Platinum
RR
Platinum\u2014Rising Rivals
SV
Platinum\u2014Supreme Victors
AR
Platinum\u2014Arceus
HS
HeartGold & SoulSilver
UL
HS\u2014Unleashed
UD
HS\u2014Undaunted
TM
HS\u2014Trium